In [1]:
import os
import pickle

# # import pickle file 
with open('../scenario-properties-controller.pkl', 'rb') as f:
    baseline = pickle.load(f)

# with open('../scenario-properties-controller_orig.pkl', 'rb') as f:
#     baseline = pickle.load(f)

/Users/glavezzi/Desktop/pyssem/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import sympy as sp

# Equations
baseline.equations

# x0 initial population
baseline.x0

# baseline.
baseline.all_symbolic_vars

# solver
equations_flattened = [baseline.equations[i, j] for j in range(baseline.equations.cols) for i in range(baseline.equations.rows)]

equations = [sp.lambdify(baseline.all_symbolic_vars, eq, 'numpy') for eq in equations_flattened]

In [5]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint

def PrfAll(x, baseline):
  """
  Python version of the PrfAll function.

  Args:
    x: Input array, where the first N_shell elements represent S_all.
    baseline: Dictionary containing data, including 'N_shell'.

  Returns:
    f: Objective function.
  """

  N_shell = baseline.n_shells
  S_all = x[:N_shell]  # Extract S_all from x

  # HIGH CAPACITY
  f = -np.sum(np.log(S_all)**2) 

  # Uncomment for NOT HIGH CAPACITY scenarios (as needed)
  # f = - np.sum(np.log(S_all[5:8])) # shells at 500-600 km (Python index starts at 0)
  f = - np.sum(S_all[6]) # shell at 550 km (Python index starts at 0)

  return f

def PrcAll(x, baseline):
    """
    Evaluates all constraints at once.

    Args:
      x: The optimization variables.
      baseline: Your baseline object containing symbolic variables.

    Returns:
      An array containing the results of all constraint function evaluations.
    """

    c_eq = np.array([sp.lambdify(baseline.all_symbolic_vars, eq, 'numpy')(*x) for eq in equations_flattened])
    
    # c_ineq 
    # y_fail_l(k) = -(lambda * Dt * (1 - failure_rate_L) - S);
    # y_fail_u(k) = lambda * Dt * (1 - failure_rate_U) - S; 

    return c_eq

## Initial guess
x0 = np.ones(len(baseline.all_symbolic_vars)) * 1

## Bounds
lb = np.ones(len(baseline.all_symbolic_vars)) * 1
ub = np.inf

## Objective function
objective = PrfAll

## Constraints for SLSQP
# con1 = {'type': 'eq', 'fun': lambda x: PrcAll(x, baseline)}
# # con2 = {'type': 'ineq', 'fun': constraint2}
# nonlcon = con1
# opt = {'ftol': 1e-12, 'eps': 1e-18, 'disp': True, 'maxiter': 5e5} 

## Constraints for trust-constr
nonlcon = NonlinearConstraint(lambda x: PrcAll(x, baseline), 0, 0)

## Perform the optimization
result = minimize(objective, x0, args=(baseline), method='trust-constr', # SLSQP, trust-constr, 
                  jac=None, hess=None, hessp=None, 
                  bounds=Bounds(lb=lb, ub=ub, keep_feasible=False), 
                  constraints=nonlcon, 
                  tol=None,
                  options=None)

## Print the results
print("Optimal found at:", result.x)
print("Function value at optimal:", -result.fun)
result

KeyboardInterrupt: 

In [7]:
equations

[<function _lambdifygenerated(S_1, S_2, S_3, S_4, S_5, S_6, S_7, S_8, S_9, S_10, S_11, S_12, S_13, S_14, Dummy_47, Dummy_41, Dummy_40, Dummy_39, Dummy_38, Dummy_37, Dummy_36, Dummy_35, Dummy_34, Dummy_46, Dummy_45, Dummy_44, Dummy_43, Dummy_42, N_200kg_1, N_200kg_2, N_200kg_3, N_200kg_4, N_200kg_5, N_200kg_6, N_200kg_7, N_200kg_8, N_200kg_9, N_200kg_10, N_200kg_11, N_200kg_12, N_200kg_13, N_200kg_14)>,
 <function _lambdifygenerated(S_1, S_2, S_3, S_4, S_5, S_6, S_7, S_8, S_9, S_10, S_11, S_12, S_13, S_14, Dummy_61, Dummy_55, Dummy_54, Dummy_53, Dummy_52, Dummy_51, Dummy_50, Dummy_49, Dummy_48, Dummy_60, Dummy_59, Dummy_58, Dummy_57, Dummy_56, N_200kg_1, N_200kg_2, N_200kg_3, N_200kg_4, N_200kg_5, N_200kg_6, N_200kg_7, N_200kg_8, N_200kg_9, N_200kg_10, N_200kg_11, N_200kg_12, N_200kg_13, N_200kg_14)>,
 <function _lambdifygenerated(S_1, S_2, S_3, S_4, S_5, S_6, S_7, S_8, S_9, S_10, S_11, S_12, S_13, S_14, Dummy_75, Dummy_69, Dummy_68, Dummy_67, Dummy_66, Dummy_65, Dummy_64, Dummy_63, Dum

In [6]:
# from scipy.optimize import show_options
# show_options('minimize', method='SLSQP')
# show_options('minimize', method='trust-constr')